In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
class CLoss(keras.losses.Loss):
    def __init__(self, threshold = 0.1, **kwargs):
        super().__init__(**kwargs)
        self.threshold = tf.constant(threshold, dtype = tf.float32)
    
    def call(self, y_true, y_pred):
        y_true = tf.constant(y_true, dtype = tf.float32)
        y_pred = tf.constant(y_pred, dtype = tf.float32)
        error = tf.abs(tf.subtract(y_true, y_pred))
        error = error[tf.greater(error, self.threshold)]
        return tf.divide(tf.reduce_sum(tf.subtract(error, self.threshold)), y_pred.shape[0])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold.numpy()}

In [3]:
closs = CLoss()
closs.get_config()

{'reduction': 'auto', 'name': None, 'threshold': 0.1}

In [4]:
closs.call([1, 2, 3, 4, 5], [1.5, 3, 2, 4, 5])

<tf.Tensor: shape=(), dtype=float32, numpy=0.43999997>

In [5]:
class CMetric(keras.metrics.Metric):
    def __init__(self, threshold = 0.1, **kwargs):
        super().__init__(**kwargs)
        self.threshold = tf.constant(threshold, dtype = tf.float32)
        self.error = self.add_weight('error', initializer = 'zeros')
        self.count = self.add_weight('count', initializer = 'zeros')
    
    def update_state(self, y_true, y_pred):
        y_true = tf.constant(y_true, dtype = tf.float32)
        y_pred = tf.constant(y_pred, dtype = tf.float32)
        
        error = tf.abs(tf.subtract(y_true, y_pred))
        error = tf.subtract(error[tf.greater(error, self.threshold)], self.threshold)
        
        self.error.assign_add(tf.reduce_sum(error))
        self.count.assign_add(y_pred.shape[0])
    
    def result(self):
        return tf.divide(self.error, self.count)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold.numpy()}

In [6]:
cmetric = CMetric()

In [7]:
cmetric.get_config()

{'name': 'c_metric', 'dtype': 'float32', 'threshold': 0.1}

In [8]:
cmetric.update_state([1,], [1.5])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.4>

In [9]:
cmetric.update_state([2,], [3,])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.65>

In [10]:
cmetric.update_state([3,], [2,])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.7333333>

In [11]:
cmetric.update_state([4, 5], [4, 5])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.43999997>

In [12]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation = None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        kself.acttivation = keras.activations.get(activations)
    
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name = "kernel", shape = [batch_input_shape[-1], self.units],
            initializer = 'glorot_normal')
        self.bias = self.add_weight(
            name = 'bian', shape = [self.units],
            initializer = 'zeros')
    
    def call(self, X):
        return self.activation(X@self.kernel+self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1]+
                             [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units': self.units,
               'activation': keras.activations.serialize(self.activation)}